# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hermanus,-34.4187,19.2345,296.37,87,37,2.79,ZA,1707011981
1,1,aitape,-3.1374,142.3475,302.09,73,95,2.31,PG,1707011981
2,2,margaret river,-33.9500,115.0667,290.94,52,75,6.58,AU,1707011981
3,3,thompson,55.7435,-97.8558,273.24,100,100,2.06,CA,1707011981
4,4,sao miguel do araguaia,-13.2750,-50.1628,296.30,87,100,0.84,BR,1707011982


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City")
# Display the map
# YOUR CODE HERE
map_plot_1    

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
narrow_data_df = city_data_df.loc[(city_data_df["Max Temp"]>21) & (city_data_df["Max Temp"]<27)]
# Drop any rows with null values
# YOUR CODE HERE
narrow_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]
# Display sample data
# YOUR CODE HERE
narrow_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]
narrow_data_df = narrow_data_df.dropna()

narrow_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,iqaluit,63.7506,-68.5145,247.42,68,0,5.14,CA,1707011983
14,14,ushuaia,-54.8000,-68.3000,282.96,66,0,1.54,AR,1707011913
19,19,khorramabad,33.4878,48.3558,270.94,93,0,2.06,IR,1707011984
20,20,villa emilio carranza,19.9667,-96.6000,296.36,70,0,3.55,MX,1707011985
22,22,sparti,37.0733,22.4297,282.01,90,0,1.78,GR,1707011985
...,...,...,...,...,...,...,...,...,...,...
544,544,ocean shores,46.9737,-124.1563,282.05,83,0,0.00,US,1707012154
550,550,gonabad,34.3529,58.6837,274.05,66,0,0.40,IR,1707012155
557,557,carloforte,39.1450,8.3058,286.52,79,0,5.06,IT,1707012157
561,561,jagannathpur,22.2333,85.6500,290.40,49,0,0.85,IN,1707012158


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""
# Display sample data
# YOUR CODE HERE
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
0,hermanus,ZA,-34.4187,19.2345,87,
1,aitape,PG,-3.1374,142.3475,73,
2,margaret river,AU,-33.9500,115.0667,52,
3,thompson,CA,55.7435,-97.8558,100,
4,sao miguel do araguaia,BR,-13.2750,-50.1628,87,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hermanus - nearest hotel: Aloe guest house
aitape - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
thompson - nearest hotel: Thompson Inn
sao miguel do araguaia - nearest hotel: Hotel São Miguel
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
blackmans bay - nearest hotel: Villa Howden
fale old settlement - nearest hotel: No hotel found
dompu - nearest hotel: No hotel found
borogontsy - nearest hotel: No hotel found
tiksi - nearest hotel: Арктика
tazovsky - nearest hotel: Тазовчанка
iqaluit - nearest hotel: Frobisher Inn
olonkinbyen - nearest hotel: No hotel found
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
paris - nearest hotel: Hôtel Esmerelda
prado - nearest hotel: Pousada Recanto do Prado
beloyarskiy - nearest hotel: Сибирь
avarua - nearest hotel: Paradise Inn
khorramabad - nearest hotel: هتل رنگین کمان
villa emilio carranza - nearest hotel: No hotel found
invercargill - nearest hotel: The Grand


,City,Country,Lat,Lng,Humidity,Hotel Name
0,hermanus,ZA,-34.4187,19.2345,87,Aloe guest house
1,aitape,PG,-3.1374,142.3475,73,No hotel found
2,margaret river,AU,-33.9500,115.0667,52,Margaret River Hotel
3,thompson,CA,55.7435,-97.8558,100,Thompson Inn
4,sao miguel do araguaia,BR,-13.2750,-50.1628,87,Hotel São Miguel
...,...,...,...,...,...,...
565,kuhmo,FI,64.1333,29.5167,96,Hotelli Kainuu
566,gerash,IR,27.6652,54.1371,54,هتل آبشار
567,lagos,NG,6.5833,3.7500,84,No hotel found
568,rabaul,PG,-4.1967,152.1721,75,Rabaul Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_2 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 500,
    size = "Humidity",
    scale = 0.6,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
# YOUR CODE HERE
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)